# Jonathan Halverson
# Wednesday, November 29, 2017
# Intro to recurrent neural networks

In [1]:
import tensorflow as tf

Time steps are handled manually.

In [2]:
n_inputs = 3
n_neurons = 5

In [3]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [4]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

In [5]:
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

In [6]:
init = tf.global_variables_initializer()

In [7]:
import numpy as np

X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t=0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t=1

In [8]:
with tf.Session() as sess:
     init.run()
     Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})

In [9]:
print(Y0_val) # output t=0

[[-0.4511241   0.99071133 -0.64638299 -0.81908041 -0.98432654]
 [-0.99963194  1.          0.9659791   0.24803112 -1.        ]
 [-0.99999988  1.          0.99987137  0.93030387 -1.        ]
 [-1.          0.99996674  0.99919569  1.         -0.99999917]]


In [10]:
print(Y1_val) # output t=1

[[-1.          1.          1.          0.99999005 -1.        ]
 [ 0.99924541  0.74179214  0.41120481  0.97355551 -0.44937527]
 [-0.99240029  1.          0.99999863  0.99996459 -1.        ]
 [ 0.78811783  0.99996871  0.99965221  0.9993192  -0.99984831]]
